In [1]:
import numpy as np
import pandas as pd

pd.options.display.max_columns = 100
idx = pd.IndexSlice

%matplotlib inline
import matplotlib.pyplot as pt
import seaborn as sb
sb.set_style('whitegrid')

import wikifunctions as wf

import requests, json, re, time, os, pprint

path = 'catlists\\'
from datetime import datetime
from bs4 import BeautifulSoup
from urllib.parse import unquote, quote
from copy import deepcopy

In [2]:
#set up an empty cat-list pair legend
#df=pd.DataFrame(columns=['category','list','suffixes'])
#df.head()
#df.to_csv(path+'keyPairs.csv',index=False)

In [3]:
def get_list(listName):
    l=listName.replace(' ','_')
    #df=pd.DataFrame()
    r = requests.get('https://en.wikipedia.org/wiki/'+str(l))
    con = r.content
    df = pd.read_html(con)
    #df.to_csv(path+l+'.csv')
    df['inList']=listName
    df['listLink']='https://en.wikipedia.org/wiki/'+str(l)
    return df

In [28]:
def standardizeLists(df):
    #name
    dz=pd.DataFrame(columns=['name'])
    try:
        dz['name']=df['Name']
    except:
        print(df.head())
        return 
    #birth and/or death years
    try:
        dz['years']=df['Dates']
    except:
        try:
            dz['years']=df['Birth year']
        except:
            pass
    #nationality
    try:
        dz['country']=df['Nationality']
    except:
        pass
    #pronouns maybe
    try:
        dz['pronouns']=df['Pronouns']
    except:
        pass
    #misc info
    try:
        dz['misc']=df['Comments']
    except:
        try:
            dz['misc']=df['Notable as']
        except:
            try:
                dz['misc']=df['Notes']
            except:
                pass
    dz['inList']=df['inList']
    dz['listLink']=df['listLink']
    
    return dz

In [29]:
def get_catMembers(categoryName, d=3):
    c=categoryName.replace(' ','_')
    catMembers = wf.get_category_members('Category:'+c,
                                        depth=d)
    df=pd.DataFrame()
    df['name']=catMembers
    df['wikiLink']=df['name'].apply(lambda s: 'https://en.wikipedia.org/wiki/'+str(s.replace(',','').replace(' ','_')))
    df['inCategory']=categoryName
    return df

In [30]:
#d=get_list('List of transgender people')
#d=standardizeLists(d)
#d.head()

In [31]:
def diffListCat(categoryName,listName,suffixes=[]):
    a=get_catMembers(categoryName)
    b=get_list(listName,suffixes)
    b=standardizeLists(b)
    try:
        c=list(set(b['name']) and set(a['name']))
        print(c)
    except:
        return
    notInList=b[b['name'] not in c]
    notInCategory=a[a['name'] not in c]
    notInList['addToList']=listName
    notInCategory['addToCategory']=categoryName
    return [notInList, notInCategory]

In [32]:
df=pd.read_csv(path+'keyPairs.csv',keep_default_na=False,index_col=False)
addToList=pd.DataFrame()
addToCat=pd.DataFrame()
for index, row in df.iterrows():
    try:
        ls,ct=diffListCat(categoryName=row[0],listName=row[1],suffixes=row[2])
        addToList.append(ls)
        addToCat.append(ct)
    except:
        pass

Empty DataFrame
Columns: [inList, listLink]
Index: []
Empty DataFrame
Columns: [inList, listLink]
Index: []
Empty DataFrame
Columns: [inList, listLink]
Index: []
Empty DataFrame
Columns: [inList, listLink]
Index: []
Empty DataFrame
Columns: [inList, listLink]
Index: []


In [ ]:
addToList.to_csv('addToList.csv',index=False)
addCategoryTag.to_csv('addToCat.csv',index=False)